In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
%matplotlib inline

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader, TensorDataset

train = pd.read_csv('../data/sensor/train.csv')
test = pd.read_csv('../data/sensor/test.csv')
train_labels = pd.read_csv("../data/sensor/train_labels.csv")

train = train.set_index(["sequence", "subject", "step"])
test = test.set_index(["sequence", "subject", "step"])

In [13]:
print("Checking if there are any missing values:")
print("Train: {}".format(train.isnull().sum().sum()))
print("Test: {}".format(test.isnull().sum().sum()))

Checking if there are any missing values:
Train: 0
Test: 0


In [14]:
def add_features(df, features):
    for feature in features:
        df_grouped = df.groupby("sequence")[feature]
        df_rolling = df_grouped.rolling(5, center=True)
        
        df[feature + "_lag1"] = df_grouped.shift(1)
        df[feature + "_diff1"] = df[feature] - df[feature + "_lag1"]
        df[feature + "_lag2"] = df_grouped.shift(2)
        df[feature + "_diff2"] = df[feature] - df[feature + "_lag2"]
        df[feature + "_roll_mean"] = df_rolling.mean().reset_index(0, drop=True)
        df[feature + "_roll_std"] = df_rolling.std().reset_index(0, drop=True)
    df.dropna(axis=0, inplace=True)
    return

features = ["sensor_{:02d}".format(i) for i in range(13)]
# add_features(train, features)
# add_features(test, features)
train.head()

sensor_00  sensor_01  sensor_02  sensor_03  sensor_04  \
sequence subject step                                                          
0        47      0     -0.196291   0.112395        1.0   0.329204  -1.004660   
                 1     -0.447450   0.134454        1.0  -0.658407   0.162495   
                 2      0.326893  -0.694328        1.0   0.330088   0.473678   
                 3      0.523184   0.751050        1.0   0.976991  -0.563287   
                 4      0.272025   1.074580        1.0  -0.136283   0.398579   

                       sensor_05  sensor_06  sensor_07  sensor_08  sensor_09  \
sequence subject step                                                          
0        47      0     -0.131638  -0.127505   0.368702       -0.1  -0.963873   
                 1      0.340314  -0.209472  -0.867176        0.2  -0.301301   
                 2      1.280479  -0.094718   0.535878        1.4   1.002168   
                 3     -0.720269   0.793260   0.951145       -0.3  -0.995665   
                 4      0.044877   0.560109  -0.541985       -0.9   1.055636   

                       sensor_10  sensor_11  sensor_12  
sequence subject step                                   
0        47      0     -0.985069   0.531893   4.751492  
                 1      0.082733  -0.231481   0.454390  
                 2      0.449221  -0.586420  -4.736147  
                 3     -0.434290   1.344650   0.429241  
                 4      0.812631   0.123457  -0.223359

In [15]:
input_size = train.shape[1]
sequence_length = len(train.index.get_level_values(2).unique())

# Scaling test and train
scaler = StandardScaler()
train = scaler.fit_transform(train)
test = scaler.transform(test)

# Reshaping:
train = train.reshape(-1, sequence_length, input_size)
test = test.reshape(-1, sequence_length, input_size)
print("After Reshape")
print("Shape of training set: {}".format(train.shape))
print("Shape of test set: {}".format(test.shape))

After Reshape
Shape of training set: (25968, 60, 13)
Shape of test set: (12218, 60, 13)


In [16]:
# Splitting train data set into train and validation sets
# validation size is selected as 0.2
t_X, v_X, t_y, v_y = train_test_split(train, train_labels.state, test_size=0.20,
                                      shuffle=True, random_state=0)

# Converting train, validation and test data into tensors
train_X_tensor = torch.tensor(t_X).float()
val_X_tensor = torch.tensor(v_X).float()
test_tensor = torch.tensor(test).float()

# Converting train and validation labels into tensors
train_y_tensor = torch.tensor(t_y.values)
val_y_tensor = torch.tensor(v_y.values)

# Creating train and validation tensors
train_tensor = TensorDataset(train_X_tensor, train_y_tensor)
val_tensor = TensorDataset(val_X_tensor, val_y_tensor)

# Defining the dataloaders
dataloaders = dict()
dataloaders["train"] = DataLoader(train_tensor, batch_size=64, shuffle=True)
dataloaders["val"] = DataLoader(val_tensor, batch_size=32)
dataloaders["test"] = DataLoader(test_tensor, batch_size=32)
print("Dataloaders are created!")

Dataloaders are created!


In [17]:
# Definition of a RNN Model class
class RNN(nn.Module):
    def __init__(self, input_size, hidden_sizes, seq_len, dropout=0.5, output_size=1):
        super(RNN, self).__init__()
        
        # LSTM Layers
        self.lstm_1 = nn.LSTM(input_size, hidden_sizes[0], num_layers=2,
                            batch_first=True, bidirectional=True, dropout=dropout)
        self.lstm_21 = nn.LSTM(2*hidden_sizes[0], hidden_sizes[1], num_layers=2,
                             batch_first=True, bidirectional=True, dropout=dropout)
        self.lstm_22 = nn.LSTM(input_size, hidden_sizes[1], num_layers=2,
                             batch_first=True, bidirectional=True, dropout=dropout)
        self.lstm_31 = nn.LSTM(2*hidden_sizes[1], hidden_sizes[2], num_layers=2,
                             batch_first=True, bidirectional=True, dropout=dropout)
        self.lstm_32 = nn.LSTM(4*hidden_sizes[1], hidden_sizes[2], num_layers=2,
                             batch_first=True, bidirectional=True, dropout=dropout)
        self.lstm_41 = nn.LSTM(2*hidden_sizes[2], hidden_sizes[3], num_layers=2,
                             batch_first=True, bidirectional=True, dropout=dropout)
        self.lstm_42 = nn.LSTM(4*hidden_sizes[2], hidden_sizes[3], num_layers=2,
                             batch_first=True, bidirectional=True, dropout=dropout)
        hidd = 2*hidden_sizes[0] + 4*(hidden_sizes[1]+hidden_sizes[2]+hidden_sizes[3])
        self.lstm_5 = nn.LSTM(hidd, hidden_sizes[4], num_layers=2,
                             batch_first=True, bidirectional=True, dropout=dropout)
        
        # Fully Connected Layer
        self.fc = nn.Sequential(nn.Linear(2*hidden_sizes[4]*seq_len, 4096),
                                nn.ReLU(inplace=True),
                                nn.Dropout(p=dropout),
                                nn.Linear(4096, 1024),
                                nn.ReLU(inplace=True),
                                nn.Dropout(p=dropout),
                                nn.Linear(1024, output_size),
                                nn.Sigmoid()
                               )
        
    def forward(self, x):
        # lstm layers:
        x1, _ = self.lstm_1(x)
        
        x_x1, _ = self.lstm_21(x1)
        x_x2, _ = self.lstm_22(x)
        x2 = torch.cat([x_x1, x_x2], dim=2)
        
        x_x1, _ = self.lstm_31(x_x1)
        x_x2, _ = self.lstm_32(x2)
        x3 = torch.cat([x_x1, x_x2], dim=2)
        
        x_x1, _ = self.lstm_41(x_x1)
        x_x2, _ = self.lstm_42(x3)
        x4 = torch.cat([x_x1, x_x2], dim=2)
        x = torch.cat([x1, x2, x3, x4], dim=2)
        x, _ = self.lstm_5(x)
        
        # fully connected layers:
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        return x

In [18]:
### VALIDATION FUNCTION
def validation(model, loader, criterion, device="cpu"):
    model.eval()
    loss = 0
    preds_all = torch.LongTensor()
    labels_all = torch.LongTensor()
    
    with torch.no_grad():
        for batch_x, labels in loader:
            labels_all = torch.cat((labels_all, labels), dim=0)
            batch_x, labels = batch_x.to(device), labels.to(device)
            labels = labels.unsqueeze(1).float()
            
            output = model.forward(batch_x)
            loss += criterion(output,labels).item()
            preds_all = torch.cat((preds_all, output.to("cpu")), dim=0)
    total_loss = loss/len(loader)
    auc_score = roc_auc_score(labels_all, preds_all)
    return total_loss, auc_score

In [19]:
### TRAINING FUNCTION
def train_model(model, trainloader, validloader, criterion, optimizer, 
                scheduler, epochs=20, device="cpu", print_every=1):
    model.to(device)
    best_auc = 0
    best_epoch = 0
    for e in range(epochs):
        model.train()
        
        for batch_x, labels in trainloader:
            batch_x, labels = batch_x.to(device), labels.to(device)
            labels = labels.unsqueeze(1).float()
            
            # Training 
            optimizer.zero_grad()
            output = model.forward(batch_x)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            scheduler.step()
        # at the end of each epoch calculate loss and auc score:
        model.eval()
        train_loss, train_auc = validation(model, trainloader, criterion, device)
        valid_loss, valid_auc = validation(model, validloader, criterion, device)
        if valid_auc > best_auc:
            best_auc = valid_auc
            best_epoch = e
            torch.save(model.state_dict(), "best-state.pt")
        if e % print_every == 0:
            to_print = "Epoch: "+str(e+1)+" of "+str(epochs)
            to_print += ".. Train Loss: {:.4f}".format(train_loss)
            to_print += ".. Valid Loss: {:.4f}".format(valid_loss)
            to_print += ".. Valid AUC: {:.3f}".format(valid_auc)
            print(to_print)
    # After Training:
    model.load_state_dict(torch.load("best-state.pt"))
    to_print = "\nTraining completed. Best state dict is loaded.\n"
    to_print += "Best Valid AUC is: {:.4f} after {} epochs".format(best_auc,best_epoch+1)
    print(to_print)
    return

In [20]:
### PREDICTION FUNCTION
def prediction(model, loader, device="cpu"):
    model.to(device)
    model.eval()
    preds_all = torch.LongTensor()
    
    with torch.no_grad():
        for batch_x in loader:
            batch_x = batch_x.to(device)
            
            output = model.forward(batch_x).to("cpu")
            preds_all = torch.cat((preds_all, output), dim=0)
    return preds_all

In [21]:
hidden_sizes = [288, 192, 144, 96, 32]
max_learning_rate = 0.001
epochs = 41

# Model
print(input_size, hidden_sizes, sequence_length)
model_lstm = RNN(input_size, hidden_sizes, sequence_length)
print("Model: ")
print(model_lstm)

# criterion, optimizer, scheduler
#criterion = nn.MSELoss()
criterion = nn.BCELoss()
optimizer = optim.Adam(model_lstm.parameters(), lr=max_learning_rate)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer,
                                          max_lr = max_learning_rate,
                                          epochs = epochs,
                                          steps_per_epoch = len(dataloaders["train"]),
                                          pct_start = 0.2,
                                          anneal_strategy = "cos")

13 [288, 192, 144, 96, 32] 60
Model: 
RNN(
  (lstm_1): LSTM(13, 288, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (lstm_21): LSTM(576, 192, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (lstm_22): LSTM(13, 192, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (lstm_31): LSTM(384, 144, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (lstm_32): LSTM(768, 144, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (lstm_41): LSTM(288, 96, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (lstm_42): LSTM(576, 96, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (lstm_5): LSTM(2304, 32, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (fc): Sequential(
    (0): Linear(in_features=3840, out_features=4096, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=4096, out_features=1024, bias=True)
    

In [23]:
# Checking if GPU is available
if torch.cuda.is_available():
    my_device = "cuda"
    print("GPU is enabled")
else:
    my_device = "cpu"
    print("No GPU :(")

train_losses = []
val_losses = []
model_lstm.to(my_device)
for epoch in range(epochs):
    train_loss = 0
    val_loss = 0
    model_lstm.train()
    for inputs, labels in dataloaders["train"]:
        inputs, labels = inputs.to(my_device), labels.to(my_device)
        labels = labels.unsqueeze(1).float()

        optimizer.zero_grad()
        output = model_lstm(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_losses.append(train_loss/len(dataloaders["train"]))
    
    model_lstm.eval()
    preds_all = torch.LongTensor()
    labels_all = torch.LongTensor()
    with torch.no_grad():
        for inputs, labels in dataloaders["val"]:
            inputs, labels = inputs.to(my_device), labels.to(my_device)
            output = model_lstm(inputs)
            preds_all = torch.cat((preds_all, output.to("cpu")), dim=0)
            labels_all = torch.cat((labels_all, labels.to("cpu")), dim=0)
            loss = criterion(output.squeeze(), labels.float())
            val_loss += loss.item()
        val_losses.append(val_loss/len(dataloaders["val"]))
        
        auc_score = roc_auc_score(labels_all, preds_all)
    print("Epoch: {}/{}.. ".format(epoch+1, epochs),
          "Training Loss: {:.3f}.. ".format(train_losses[-1]),
          "Validation Loss: {:.3f}.. ".format(val_losses[-1]),
          "Validation AUC: {:.3f}".format(auc_score))

# Training
# train_model(model = model_lstm,
#             trainloader = dataloaders["train"],
#             validloader = dataloaders["val"],
#             criterion = criterion,
#             optimizer = optimizer,
#             scheduler = scheduler,
#             epochs = epochs,
#             device = my_device,
#             print_every = 2)

GPU is enabled
Epoch: 1/41..  Training Loss: 0.684..  Validation Loss: 0.681..  Validation AUC: 0.588
Epoch: 2/41..  Training Loss: 0.662..  Validation Loss: 0.640..  Validation AUC: 0.714
Epoch: 3/41..  Training Loss: 0.586..  Validation Loss: 0.531..  Validation AUC: 0.812
Epoch: 4/41..  Training Loss: 0.515..  Validation Loss: 0.472..  Validation AUC: 0.858
Epoch: 5/41..  Training Loss: 0.476..  Validation Loss: 0.448..  Validation AUC: 0.875
Epoch: 6/41..  Training Loss: 0.447..  Validation Loss: 0.422..  Validation AUC: 0.888
Epoch: 7/41..  Training Loss: 0.430..  Validation Loss: 0.405..  Validation AUC: 0.897
Epoch: 8/41..  Training Loss: 0.418..  Validation Loss: 0.401..  Validation AUC: 0.899
Epoch: 9/41..  Training Loss: 0.408..  Validation Loss: 0.401..  Validation AUC: 0.900
Epoch: 10/41..  Training Loss: 0.401..  Validation Loss: 0.408..  Validation AUC: 0.904
Epoch: 11/41..  Training Loss: 0.400..  Validation Loss: 0.421..  Validation AUC: 0.900
Epoch: 12/41..  Training L

KeyboardInterrupt: 

In [24]:
y_pred = prediction(model_lstm, dataloaders["test"], device=my_device)
print("Prediction completed, first 5 states:")
y_pred[:5]

Prediction completed, first 5 states:


tensor([[0.9915],
        [0.9708],
        [0.0377],
        [0.9311],
        [0.6453]])

In [27]:
submission = pd.read_csv("../data/sensor/sample_submission.csv")
submission.state = torch.round(y_pred)
submission.head()

,sequence,state
0,25968,1.0
1,25969,1.0
2,25970,0.0
3,25971,1.0
4,25972,1.0


In [28]:
submission.to_csv('submission.csv', index=False)
print("Resuls are saved to submission.csv")

Resuls are saved to submission.csv
